# Title

#### RESEARCH QUESTION

Is DC area bicycle ridership consistent along the entire year?

#### OBTAINING DATA

We had obtained data from multiple sources which was comprised of a sunset-sunrise API and CSV files for Capital Bike Share rentals.  The CSV files are publicly available for downloaded and contain information on their bike rentals from members and "casual" users.  The sunrise-sunset API allows you to request information pertaining to the duration of daylight ("Day-length") by a given day or week or by geo-location.  Day-length is expressed in seconds.

#### DATA INGESTION

The Capital Bike Share rental data contains every recorded instance of a bike rental by "Member" and "Casual" users which include time duration of the rental, start and end date, start and end location with an ID number, and a bike number.  We obtained data sheets for each month in 2020 except for April which was the month that Capital Bike Share rental was shutdown.

The sunrise-sunset API provides information pertaining to the duration of sunlight on a given day or week.  This information may be requested by latitude and longtitude coordinates and dates.

In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import time
import json

In [2]:
jan20bikedata=pd.read_csv("202001-capitalbikeshare-tripdata.csv")
feb20bikedata=pd.read_csv("202002-capitalbikeshare-tripdata.csv")
mar20bikedata=pd.read_csv("202003-capitalbikeshare-tripdata.csv")
may20bikedata=pd.read_csv("202005-capitalbikeshare-tripdata.csv")
jun20bikedata=pd.read_csv("202006-capitalbikeshare-tripdata.csv")
jul20bikedata=pd.read_csv("202007-capitalbikeshare-tripdata.csv")
aug20bikedata=pd.read_csv("202008-capitalbikeshare-tripdata.csv")
sep20bikedata=pd.read_csv("202009-capitalbikeshare-tripdata.csv")
oct20bikedata=pd.read_csv("202010-capitalbikeshare-tripdata.csv")
nov20bikedata=pd.read_csv("202011-capitalbikeshare-tripdata.csv")
dec20bikedata=pd.read_csv("202012-capitalbikeshare-tripdata.csv")

In [3]:
jan20bikedata

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,506,2020-01-01 00:00:59,2020-01-01 00:09:26,31623,Columbus Circle / Union Station,31658,Rosedale Rec Center,W22189,Member
1,1050,2020-01-01 00:01:16,2020-01-01 00:18:47,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31606,Potomac & Pennsylvania Ave SE,W21067,Member
2,1687,2020-01-01 00:02:42,2020-01-01 00:30:50,31218,L'Enfant Plaza / 7th & C St SW,31271,Constitution Ave & 2nd St NW/DOL,W24359,Member
3,182,2020-01-01 00:02:44,2020-01-01 00:05:47,31627,3rd & M St NE,31508,Gallaudet / 8th St & Florida Ave NE,W23334,Member
4,1002,2020-01-01 00:06:12,2020-01-01 00:22:54,31247,Jefferson Dr & 14th St SW,31274,10th & G St NW,W23441,Member
...,...,...,...,...,...,...,...,...,...
196964,1279,2020-01-31 23:56:42,2020-02-01 00:18:02,31324,18th & New Hampshire Ave NW,31205,21st & I St NW,W20938,Member
196965,561,2020-01-31 23:57:19,2020-02-01 00:06:40,31204,20th & E St NW,31225,C & O Canal & Wisconsin Ave NW,W20045,Member
196966,360,2020-01-31 23:58:51,2020-02-01 00:04:51,32048,Bethesda Metro / Wisconsin Ave & Old Georgetow...,32028,Norfolk & Rugby Ave,W00517,Member
196967,1491,2020-01-31 23:59:15,2020-02-01 00:24:07,31505,Eckington Pl & Q St NE,31417,Kansas Ave & Sherman Cr NW,W21859,Member


#### DATA CLEANING

For the months of January through March, there is a column titled "Duration" which represented the duration of the bike rental expressed in seconds.  This data was not found in the rest of the months for 2020.

In [4]:
jun20bikedata

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,74DB76AD6F43E842,docked_bike,2020-06-25 11:17:16,2020-06-25 11:32:09,16th & R St NW,31282.0,Park Rd & Holmead Pl NW,31602.0,38.912652,-77.036278,38.930800,-77.031500,member
1,8B65EBD8A01E8D4A,docked_bike,2020-06-07 16:51:39,2020-06-07 18:10:57,Prince George's Plaza Metro,32420.0,Prince George's Plaza Metro,32420.0,38.965742,-76.954803,38.965742,-76.954803,casual
2,8D82B9732C0A960B,docked_bike,2020-06-19 09:59:23,2020-06-19 11:17:19,Capitol Heights Metro,32419.0,Capitol Heights Metro,32419.0,38.888524,-76.913188,38.888524,-76.913188,casual
3,05A655CC02273BFF,docked_bike,2020-06-02 09:00:39,2020-06-02 09:55:28,Wilson Blvd & N Franklin Rd,31020.0,Wilson Blvd & N Franklin Rd,31020.0,38.890400,-77.088900,38.890400,-77.088900,casual
4,0CEBC63E774016F0,docked_bike,2020-06-21 14:09:11,2020-06-21 14:16:55,14th & Girard St NW,31123.0,18th & R St NW,31278.0,38.925284,-77.032375,38.912648,-77.041834,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214042,25C1E2E585C75BF3,docked_bike,2020-06-21 18:31:26,2020-06-21 18:57:03,3rd & H St NE,31616.0,11th & S St NW,31280.0,38.900412,-77.001949,38.913761,-77.027025,casual
214043,7E2BF68F9CFB13F1,docked_bike,2020-06-29 01:01:10,2020-06-29 01:10:41,7th & R St NW / Shaw Library,31245.0,11th & S St NW,31280.0,38.912719,-77.022155,38.913761,-77.027025,member
214044,223A071C4B5AF02C,docked_bike,2020-06-09 20:27:07,2020-06-09 20:37:54,1st & M St NE,31603.0,11th & S St NW,31280.0,38.905700,-77.005600,38.913761,-77.027025,casual
214045,8B57E6DAB8CBFD15,docked_bike,2020-06-13 07:38:43,2020-06-13 07:50:57,7th & R St NW / Shaw Library,31245.0,11th & S St NW,31280.0,38.912719,-77.022155,38.913761,-77.027025,member


In [5]:
jan20bikedata=jan20bikedata[['Duration','Start date','End date','Member type']]
jan20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']

feb20bikedata=feb20bikedata[['Duration','Start date','End date','Member type']]
feb20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']

mar20bikedata=mar20bikedata[['Duration','Start date','End date','Member type']]
mar20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']

The only way to obtain the duration value was to parse the time found in the "started_at" and "ended_at" columns, create new columns for those times titled "start" and "end", create a new column titled "Duration" which would then obtain the final value by subtracting the "end" time from the "start" time.  The "start" and "end" times were converted to datetime objects and then the final value was converted to seconds.

In [6]:
may20bikedata=may20bikedata[['started_at','ended_at','member_casual']]
may20bikedata.loc[:,'start']=may20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
may20bikedata.loc[:,'end']=may20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
may20bikedata.loc[:,'duration'] = pd.to_datetime(may20bikedata['end']) - pd.to_datetime(may20bikedata['start'])
may20bikedata.loc[:,'duration'] = may20bikedata['duration'].apply(lambda x: x.total_seconds())

In [7]:
may20bikedata

,started_at,ended_at,member_casual,start,end,duration
0,2020-05-30 17:25:29,2020-05-31 18:25:22,casual,17:25:29,18:25:22,3593.0
1,2020-05-09 14:42:04,2020-05-09 15:06:33,member,14:42:04,15:06:33,1469.0
2,2020-05-24 17:27:19,2020-05-24 17:43:51,member,17:27:19,17:43:51,992.0
3,2020-05-27 15:29:52,2020-05-27 15:47:13,casual,15:29:52,15:47:13,1041.0
4,2020-05-31 14:06:03,2020-05-31 14:30:30,casual,14:06:03,14:30:30,1467.0
...,...,...,...,...,...,...
149046,2020-05-12 08:15:31,2020-05-12 08:44:15,member,08:15:31,08:44:15,1724.0
149047,2020-05-17 13:19:33,2020-05-17 13:39:05,casual,13:19:33,13:39:05,1172.0
149048,2020-05-30 14:53:00,2020-05-30 15:04:16,casual,14:53:00,15:04:16,676.0
149049,2020-05-27 17:09:42,2020-05-27 17:31:43,casual,17:09:42,17:31:43,1321.0


In [8]:
jun20bikedata=jun20bikedata[['started_at','ended_at','member_casual']]
jun20bikedata.loc[:,'start']=jun20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
jun20bikedata.loc[:,'end']=jun20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
jun20bikedata.loc[:,'duration'] = pd.to_datetime(jun20bikedata['end']) - pd.to_datetime(jun20bikedata['start'])
jun20bikedata.loc[:,'duration'] = jun20bikedata['duration'].apply(lambda x: x.total_seconds())

c:\users\miguel\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
c:\users\miguel\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [9]:
jul20bikedata=jul20bikedata[['started_at','ended_at','member_casual']]
jul20bikedata.loc[:,'start']=jul20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
jul20bikedata.loc[:,'end']=jul20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
jul20bikedata.loc[:,'duration'] = pd.to_datetime(jul20bikedata['end']) - pd.to_datetime(jul20bikedata['start'])
jul20bikedata.loc[:,'duration'] = jul20bikedata['duration'].apply(lambda x: x.total_seconds())

In [10]:
aug20bikedata=aug20bikedata[['started_at','ended_at','member_casual']]
aug20bikedata.loc[:,'start']=aug20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
aug20bikedata.loc[:,'end']=aug20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
aug20bikedata.loc[:,'duration'] = pd.to_datetime(aug20bikedata['end']) - pd.to_datetime(aug20bikedata['start'])
aug20bikedata.loc[:,'duration'] = aug20bikedata['duration'].apply(lambda x: x.total_seconds())

In [11]:
sep20bikedata=sep20bikedata[['started_at','ended_at','member_casual']]
sep20bikedata.loc[:,'start']=sep20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
sep20bikedata.loc[:,'end']=sep20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
sep20bikedata.loc[:,'duration'] = pd.to_datetime(sep20bikedata['end']) - pd.to_datetime(sep20bikedata['start'])
sep20bikedata.loc[:,'duration'] = sep20bikedata['duration'].apply(lambda x: x.total_seconds())

In [12]:
oct20bikedata=oct20bikedata[['started_at','ended_at','member_casual']]
oct20bikedata.loc[:,'start']=oct20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
oct20bikedata.loc[:,'end']=oct20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
oct20bikedata.loc[:,'duration'] = pd.to_datetime(oct20bikedata['end']) - pd.to_datetime(oct20bikedata['start'])
oct20bikedata.loc[:,'duration'] = oct20bikedata['duration'].apply(lambda x: x.total_seconds())

In [13]:
nov20bikedata=nov20bikedata[['started_at','ended_at','member_casual']]
nov20bikedata.loc[:,'start']=nov20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
nov20bikedata.loc[:,'end']=nov20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
nov20bikedata.loc[:,'duration'] = pd.to_datetime(nov20bikedata['end']) - pd.to_datetime(nov20bikedata['start'])
nov20bikedata.loc[:,'duration'] = nov20bikedata['duration'].apply(lambda x: x.total_seconds())

In [14]:
dec20bikedata=dec20bikedata[['started_at','ended_at','member_casual']]
dec20bikedata.loc[:,'start']=dec20bikedata['started_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
dec20bikedata.loc[:,'end']=dec20bikedata['ended_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%H:%M:%S'))
dec20bikedata.loc[:,'duration'] = pd.to_datetime(dec20bikedata['end']) - pd.to_datetime(dec20bikedata['start'])
dec20bikedata.loc[:,'duration'] = dec20bikedata['duration'].apply(lambda x: x.total_seconds())

In [17]:
may20bikedata=may20bikedata[['duration','started_at','ended_at','member_casual']]
may20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
jun20bikedata=jun20bikedata[['duration','started_at','ended_at','member_casual']]
jun20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
jul20bikedata=jul20bikedata[['duration','started_at','ended_at','member_casual']]
jul20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
aug20bikedata=aug20bikedata[['duration','started_at','ended_at','member_casual']]
aug20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
sep20bikedata=sep20bikedata[['duration','started_at','ended_at','member_casual']]
sep20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
oct20bikedata=oct20bikedata[['duration','started_at','ended_at','member_casual']]
oct20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
nov20bikedata=nov20bikedata[['duration','started_at','ended_at','member_casual']]
nov20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']
dec20bikedata=dec20bikedata[['duration','started_at','ended_at','member_casual']]
dec20bikedata.columns=['Duration (seconds)','Start date','End date','Member type']


Once the months of May through December were cleaned, the columns for all of the months were re-ordered and sorted only with the necessary information for our analysis.  From here we needed to group the information by the member type in which we would eventually use to combine to find total values from both member types.

Data from the sunrise-sunset API and from Capital Bike Share rentals expressed their duration values in seconds and we therefore converted those values to minutes to make calculations easier for the analysis.  We then created a new column titled "date" which we filled by parsing the "Start date" column and formatting that value to only show the date in the new column.  The data was then grouped by this date and the value for "Duration" was aggregated to find the sum of the durations for each day of the month.

In [18]:
jan20grouped=jan20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
feb20grouped=feb20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
mar20grouped=mar20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
may20grouped=may20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
jun20grouped=jun20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
jul20grouped=jul20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
aug20grouped=aug20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
sep20grouped=sep20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
oct20grouped=oct20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
nov20grouped=nov20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()
dec20grouped=dec20bikedata.groupby('Member type')['Duration (seconds)'].agg('sum').reset_index()


In [19]:
jan20grouped.loc[:,'Total minutes'] = round(jan20grouped['Duration (seconds)']/60,2)
feb20grouped.loc[:,'Total minutes'] = round(feb20grouped['Duration (seconds)']/60,2)
mar20grouped.loc[:,'Total minutes'] = round(mar20grouped['Duration (seconds)']/60,2)
may20grouped.loc[:,'Total minutes'] = round(may20grouped['Duration (seconds)']/60,2)
jun20grouped.loc[:,'Total minutes'] = round(jun20grouped['Duration (seconds)']/60,2)
jul20grouped.loc[:,'Total minutes'] = round(jul20grouped['Duration (seconds)']/60,2)
aug20grouped.loc[:,'Total minutes'] = round(aug20grouped['Duration (seconds)']/60,2)
sep20grouped.loc[:,'Total minutes'] = round(sep20grouped['Duration (seconds)']/60,2)
oct20grouped.loc[:,'Total minutes'] = round(oct20grouped['Duration (seconds)']/60,2)
nov20grouped.loc[:,'Total minutes'] = round(nov20grouped['Duration (seconds)']/60,2)
dec20grouped.loc[:,'Total minutes'] = round(dec20grouped['Duration (seconds)']/60,2)


In [20]:
jan20bikedata.loc[:,'date'] = jan20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
feb20bikedata.loc[:,'date'] = feb20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
mar20bikedata.loc[:,'date'] = mar20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
may20bikedata.loc[:,'date'] = may20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
jun20bikedata.loc[:,'date'] = jun20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
jul20bikedata.loc[:,'date'] = jul20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
aug20bikedata.loc[:,'date'] = aug20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
sep20bikedata.loc[:,'date'] = sep20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
oct20bikedata.loc[:,'date'] = oct20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
nov20bikedata.loc[:,'date'] = nov20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
dec20bikedata.loc[:,'date'] = dec20bikedata['Start date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))


In [21]:
jan20bikedata

,Duration (seconds),Start date,End date,Member type,date
0,506,2020-01-01 00:00:59,2020-01-01 00:09:26,Member,2020-01-01
1,1050,2020-01-01 00:01:16,2020-01-01 00:18:47,Member,2020-01-01
2,1687,2020-01-01 00:02:42,2020-01-01 00:30:50,Member,2020-01-01
3,182,2020-01-01 00:02:44,2020-01-01 00:05:47,Member,2020-01-01
4,1002,2020-01-01 00:06:12,2020-01-01 00:22:54,Member,2020-01-01
...,...,...,...,...,...
196964,1279,2020-01-31 23:56:42,2020-02-01 00:18:02,Member,2020-01-31
196965,561,2020-01-31 23:57:19,2020-02-01 00:06:40,Member,2020-01-31
196966,360,2020-01-31 23:58:51,2020-02-01 00:04:51,Member,2020-01-31
196967,1491,2020-01-31 23:59:15,2020-02-01 00:24:07,Member,2020-01-31


In [22]:
datejan20=jan20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datefeb20=feb20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datemar20=mar20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datemay20=may20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datejun20=jun20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datejul20=jul20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
dateaug20=aug20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datesep20=sep20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
dateoct20=oct20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datenov20=nov20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()
datedec20=dec20bikedata.groupby('date')['Duration (seconds)'].agg('sum').reset_index()


#### DATA WRANGLING

In [98]:
sunriseAPI=[]
sunriseAPIjan=[]
sunriseAPIfeb=[]
sunriseAPImar=[]
sunriseAPImay=[]
sunriseAPIjun=[]
sunriseAPIjul=[]
sunriseAPIaug=[]
sunriseAPIsep=[]
sunriseAPIoct=[]
sunriseAPInov=[]
sunriseAPIdec=[]


The sunrise-sunset API only allows visitors to obtain historical data by one given day.  Given that we needed this data for every day for every month in the Capital Bike Share data sets, we decided to create multiple loops which would parse through each day of every month.  That value was used to obtain the duration value from the sunrise-sunset API.  The results of the API request needed to be transformed to a JSON object and normalized which stored the results of the API call into a new Dataframe called "month".  

The duration value was stored into a variable called "day" which would aggregate every duration value for every day of the month.  The total value was appended to a list named after the month.  At the same time, a separate list was obtaining the duration value which will be be used to create pandas Series objects to be used to later plot any possible correlation.

In [99]:
janlst=[]
feblst=[]
marlst=[]
maylst=[]
junlst=[]
jullst=[]
auglst=[]
seplst=[]
octlst=[]
novlst=[]
declst=[]
count=0
day=0
for num in datejan20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIjan.append(day)
    count+=1
janlst.append(day)
janSer=pd.Series(sunriseAPIjan)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


In [100]:
count=0
day=0
for num in datefeb20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIfeb.append(day)
    count+=1
feblst.append(day)
febSer=pd.Series(sunriseAPIfeb)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28


In [101]:
count=0
day=0
for num in datemar20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPImar.append(day)
    count+=1
marlst.append(day)
marSer=pd.Series(sunriseAPImar)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


In [102]:
count=0
day=0
for num in datemay20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPImay.append(day)
    count+=1
maylst.append(day)
maySer=pd.Series(sunriseAPImay)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


In [103]:
count=0
day=0
for num in datejun20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIjun.append(day)
    count+=1
junlst.append(day)
junSer=pd.Series(sunriseAPIjun)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29


In [104]:
count=0
day=0
for num in datejul20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIjul.append(day)
    count+=1
jullst.append(day)
julSer=pd.Series(sunriseAPIjul)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


In [105]:
count=0
day=0
for num in dateaug20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIaug.append(day)
    count+=1
auglst.append(day)
augSer=pd.Series(sunriseAPIaug)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


In [106]:
count=0
day=0
for num in datesep20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIsep.append(day)
    count+=1
seplst.append(day)
sepSer=pd.Series(sunriseAPIsep)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29


In [107]:
count=0
day=0
for num in dateoct20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIoct.append(day)
    count+=1
octlst.append(day)
octSer=pd.Series(sunriseAPIoct)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


In [108]:
count=0
day=0
for num in datenov20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPInov.append(day)
    count+=1
novlst.append(day)
novSer=pd.Series(sunriseAPInov)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29


In [109]:
count=0
day=0
for num in datedec20['date']:
    sunsetrise=f"https://api.sunrise-sunset.org/json?lat=38.9072&lng=-77.0369&date={num}&formatted=0"
    #time.sleep(0.15)
    test=requests.get(sunsetrise)
    print(test,count)
    month = json.loads(test.text)
    month = pd.json_normalize(month['results'])
    day+=month['day_length'].iloc[0]
    sunriseAPIdec.append(day)
    count+=1
declst.append(day)
decSer=pd.Series(sunriseAPIdec)

<Response [200]> 0
<Response [200]> 1
<Response [200]> 2
<Response [200]> 3
<Response [200]> 4
<Response [200]> 5
<Response [200]> 6
<Response [200]> 7
<Response [200]> 8
<Response [200]> 9
<Response [200]> 10
<Response [200]> 11
<Response [200]> 12
<Response [200]> 13
<Response [200]> 14
<Response [200]> 15
<Response [200]> 16
<Response [200]> 17
<Response [200]> 18
<Response [200]> 19
<Response [200]> 20
<Response [200]> 21
<Response [200]> 22
<Response [200]> 23
<Response [200]> 24
<Response [200]> 25
<Response [200]> 26
<Response [200]> 27
<Response [200]> 28
<Response [200]> 29
<Response [200]> 30


Once all of the data was obtained by the API, new dataframes were created to place the duration values for each member type of Capital Bike Share services.  These values would then be used to conduct our analysis.

In [110]:
allmonths=["JANUARY","FEBRUARY","MARCH","MAY","JUNE","JULY","AUGUST","SEPTEMBER","OCTOBER","NOVEMBER","DECEMBER"]
alltimes=[janlst[0],feblst[0],marlst[0],maylst[0],junlst[0],jullst[0],
          auglst[0],seplst[0],octlst[0],novlst[0],declst[0]]
casual=[jan20grouped['Total minutes'].iloc[0],feb20grouped['Total minutes'].iloc[0],mar20grouped['Total minutes'].iloc[0],
              may20grouped['Total minutes'].iloc[0],jun20grouped['Total minutes'].iloc[0],jul20grouped['Total minutes'].iloc[0],
              aug20grouped['Total minutes'].iloc[0],sep20grouped['Total minutes'].iloc[0],oct20grouped['Total minutes'].iloc[0],
              nov20grouped['Total minutes'].iloc[0],dec20grouped['Total minutes'].iloc[0]]
members=[jan20grouped['Total minutes'].iloc[1],feb20grouped['Total minutes'].iloc[1],mar20grouped['Total minutes'].iloc[1],
              may20grouped['Total minutes'].iloc[1],jun20grouped['Total minutes'].iloc[1],jul20grouped['Total minutes'].iloc[1],
              aug20grouped['Total minutes'].iloc[1],sep20grouped['Total minutes'].iloc[1],oct20grouped['Total minutes'].iloc[1],
              nov20grouped['Total minutes'].iloc[1],dec20grouped['Total minutes'].iloc[1]]

df=pd.DataFrame({"months":allmonths,"total seconds":alltimes,"casual bikers":casual,"bikeshare members":members})

In [111]:
df.loc[:,'total daylight']=round(df['total seconds']/60,2)
df.loc[:,'total bikers']=round(df['casual bikers']+df['bikeshare members'],2)

In [112]:
df=df[['months','casual bikers','bikeshare members','total daylight','total bikers']]
df

,months,casual bikers,bikeshare members,total daylight,total bikers
0,JANUARY,384509.47,2301011.80,18243.13,2685521.27
1,FEBRUARY,383844.93,2064545.83,18765.52,2448390.76
2,MARCH,721934.43,2362702.57,22359.10,3084637.00
3,MAY,2373519.78,1085444.18,26678.45,3458963.96
4,JUNE,2499052.30,1255758.05,26741.37,3754810.35
5,JULY,1776680.33,1085091.48,27075.40,2861771.81
6,AUGUST,1906861.92,1294970.33,25302.92,3201832.25
7,SEPTEMBER,2170666.20,1388339.85,22298.42,3559006.05
8,OCTOBER,1896417.77,1370481.40,20699.57,3266899.17
9,NOVEMBER,1444416.22,1086499.43,18062.13,2530915.65
